# Insalling Packages

In [1]:
!pip install transformers faiss-gpu huggingface weaviate-client pypdf langchain langchain_community chainlit sentence-transformers bitsandbytes accelerate --quiet

In [2]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
import chainlit as cl
from langchain.text_splitter import CharacterTextSplitter
from weaviate.embedded import EmbeddedOptions
from langchain_community.vectorstores import Weaviate
from langchain.prompts import ChatPromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import HuggingFaceHub
from langchain.embeddings import OpenAIEmbeddings
import os



#Reading PDF

In [3]:
loader = PyPDFLoader("/content/natural-language-processing-with-transformers-revised-edition-1098136799-9781098136796-9781098103248.pdf")
pages = loader.load_and_split()[6:-2]

 **Splitting text to chunks**

In [4]:
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(pages)

**Embedding Chunks**

In [5]:
Embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                    model_kwargs={'device': 'cuda'})

# Embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)

store = FAISS.from_texts([str(chunk) for chunk in chunks], Embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


**Prompt**

In [6]:
prompt = """
    Use the following pieces of information to answer the user’s question.
    If you don’t know the answer, just say that you don’t know, don’t try to make up an answer.

    Context: {context}
    Question: {question}

    Only return the helpful answer below and nothing else.
    Helpful and Caring answer:\n
    """
prompt = ChatPromptTemplate.from_template(prompt)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\n    Use the following pieces of information to answer the user’s question.\n    If you don’t know the answer, just say that you don’t know, don’t try to make up an answer.\n\n    Context: {context}\n    Question: {question}\n\n    Only return the helpful answer below and nothing else.\n    Helpful and Caring answer:\n\n    '))]


#Model Quantization

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
#load_in_4bit=True,
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16
)

**HuggingFace Login**

#Llama Model

In [8]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.c

In [9]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

model_id = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map='cuda')
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
hf = HuggingFacePipeline(pipeline=pipe)

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

**Testing Model Pipeline**

In [10]:
pipe("what is machine learning")

[{'generated_text': 'what is machine learning and its applications. everybody wants to know about it, and it\'s not hard to see why. Machine learning is a subset of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable machines to learn from data, make decisions, and improve their performance over time.\n\nIn this article, we will explore the concept of machine learning, its types, applications, and the future of this technology.\n\nWhat is Machine Learning?\n\nMachine learning is a type of AI that enables machines to learn from data without being explicitly programmed. The term "machine learning" was coined in the 1980s, and since then, it has evolved into a powerful tool for businesses and organizations to automate tasks, make predictions, and improve decision-making processes.\n\nMachine learning algorithms are designed to recognize patterns in data and learn from it. The more data the algorithm is exposed to, the better it can make predic

#RAG

In [11]:
def retrieval_qa_chain(llm, prompt, db):
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=store.as_retriever(search_kwargs={'k': 2}),
                                       return_source_documents=True,
                                       chain_type_kwargs={'prompt': prompt}
                                       )
    return qa_chain


qa = retrieval_qa_chain(hf, prompt, store)

In [12]:
def qa_bot():
    embeddings = Embeddings
    db = store
    qa_prompt = prompt
    qa = retrieval_qa_chain(hf, qa_prompt, db)
    return qa

In [13]:
def final_result(query):
    qa_result = qa_bot()
    response = qa_result({'query': query})
    return response

In [17]:
x1 = final_result("Explain encoder-decoder architectures")
x2 = final_result("How to load the emotion dataset with the load_dataset() function")

In [18]:
print(x1["query"])
print("="*20)
print(x1["result"])
print("="*20)
print(x1["source_documents"])

Explain encoder-decoder architectures
 Encoder-decoder architectures are a type of neural network design used in natural language processing (NLP) tasks. They consist of two main components: an encoder and a decoder.

     The encoder takes in a sequence of tokens (e.g. words or characters) and outputs a sequence of hidden states that capture the context and meaning of the input sequence. These hidden states are then passed to the decoder, which generates the output sequence one token at a time.

     The decoder uses the hidden states from the encoder to predict the next token in the output sequence. It does this by attending to all of the hidden states from the encoder, and assigning a different amount of attention to each one. This allows the decoder to selectively focus on different parts of the input sequence as it generates the output.

     Encoder-decoder architectures are particularly useful for tasks like machine translation, where the input sequence is long and the output se

In [19]:
print(x2["query"])
print("="*20)
print(x2["result"])
print("="*20)
print(x2["source_documents"])

How to load the emotion dataset with the load_dataset() function
 You can load the emotion dataset with the load_dataset() function by passing the name of the dataset as an argument to the function. In this case, the name of the dataset is "emotion".

Explanation:

* The emotion dataset is a pre-existing dataset available on the Hugging Face Hub.
* To load the emotion dataset, you can use the load_dataset() function and pass the name of the dataset as an argument.
* The load_dataset() function will then return a Dataset object that contains the data from the dataset.

Note:

* The load_dataset() function can also be used to load other datasets available on the Hugging Face Hub.
* You can find a list of all the available datasets by running the list_datasets() function.
[Document(page_content='page_content=\'A First Look at Hugging Face Datasets\\nWe will use \\n  Datasets to download the data from the Hugging Face Hub . We can\\nuse the list_datasets()  function to see what datasets ar

In [21]:
x3 = final_result("Explain Text Classification chapter")
print(x3["query"])
print("="*20)
print(x3["result"])
print("="*20)
print(x3["source_documents"])

Explain Text Classification chapter
 Text Classification is a task in Natural Language Processing (NLP) that involves categorizing text into predefined categories or labels. This chapter discusses the basics of text classification, including the different types of classification, the importance of preprocessing, and the various algorithms used for classification.

     The chapter also covers the challenges of text classification, such as dealing with imbalanced datasets and the need for careful evaluation metrics. Additionally, the chapter provides examples of how text classification can be applied in real-world scenarios, such as sentiment analysis and spam filtering.

     Overall, the chapter provides a comprehensive introduction to text classification and its applications in NLP.
[Document(page_content="page_content='CHAPTER 2\\nText Classification\\nText classification is one of the most common tasks in NLP; it can be used for a broad\\nrange of applications, such as tagging cust

In [22]:
x4 = final_result("What is linear combination equation")
print(x4["query"])
print("="*20)
print(x4["result"])
print("="*20)
print(x4["source_documents"])

What is linear combination equation
 Linear combination equation is a mathematical expression that combines two or more linear expressions using addition or multiplication. In the context of transformers, linear combination is used to compute the output of a layer by combining the outputs of multiple linear transformations. The linear combination equation can be written as:

    y = a1*x1 + a2*x2 +... + an*xn

where x1, x2,..., xn are the inputs to the linear combination, and a1, a2,..., an are the weights of the linear combination. The weights can be learned during training using various optimization techniques.

I hope this helps! Let me know if you have any other questions.
[Document(page_content="page_content='8A. Katharopoulos et al., “Transformers Are RNNs: Fast Autoregressive Transformers with Linear Attention” ,\\n(2020); K. Choromanski et al., “Rethinking Attention with Performers” , (2020).The trick behind linearized attention mechanisms is to express the similarity function\

In [23]:
x4 = final_result("What is Scaled dot-product attention")
print(x4["query"])
print("="*20)
print(x4["result"])
print("="*20)
print(x4["source_documents"])

What is Scaled dot-product attention
 Scaled dot-product attention is a type of attention mechanism used in Transformer models. It is called "scaled" because the attention scores are first multiplied by a scaling factor to normalize their variance, and then normalized with a softmax to ensure all the column values sum to 1. This normalization step helps to stabilize the training process by preventing large attention scores from dominating the computation.

The basic idea of scaled dot-product attention is to compute the attention scores between two vectors (the query and the key) using a dot product. However, the dot product can produce arbitrarily large numbers, which can destabilize the training process. To address this issue, the attention scores are first multiplied by a scaling factor, which normalizes their variance. Then, the scores are normalized with a softmax to ensure that all the column values sum to 1.

In more detail, the computation of scaled dot-product attention involv